In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
#downloading VGG16 weight
vggmodel = VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 244, 3) )

In [3]:
#freezing all layer
for layer in vggmodel.layers:
    layer.trainable = False

In [4]:
# # creating Fully connected layers
def FClayer(model):
    newmodel = model.output
    newmodel = Flatten()(newmodel)
    newmodel = Dense(units = 256, activation = 'relu')(newmodel)
    newmodel = Dense(units = 128, activation = 'relu')(newmodel)
    newmodel = Dense(units = 32, activation = 'relu')(newmodel)
    newmodel = Dense(units = 3, activation = 'softmax')(newmodel)
    return newmodel


In [5]:
# # Adding FClayer to VGG
from keras.layers import Dense, Flatten
from keras.models import Model

head = FClayer(vggmodel)

vggmodel = Model(inputs = vggmodel.input, outputs = head)

vggmodel.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 244, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 122, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 122, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 122, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 61, 128)       0   

# ## Compiling model

In [6]:
from keras.optimizers import RMSprop
vggmodel.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

In [7]:
# ## Augmenting Images and traing the model
from keras_preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        '/Users/vikaskumar/Documents/mlops/mlopsPractice/FaceRecogVGGTransferLearning-master/transfer learning/data/data/train',
        target_size=(224, 244),
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        '/Users/vikaskumar/Documents/mlops/mlopsPractice/FaceRecogVGGTransferLearning-master/transfer learning/data/data/test',
        target_size=(224, 244),
        class_mode='categorical')
vggmodel.fit(
        training_set,
        epochs=5,
        validation_data=test_set,
        )

vggmodel.save('vgg_face_recog.h5')

Found 252 images belonging to 3 classes.
Found 62 images belonging to 3 classes.
Epoch 1/5
8/8 [==============================] - 151s 19s/step - loss: 5.3244 - accuracy: 0.4921 - val_loss: 0.0016 - val_accuracy: 0.9839
Epoch 2/5
8/8 [==============================] - 125s 16s/step - loss: 0.0022 - accuracy: 1.0000 - val_loss: 0.0770 - val_accuracy: 0.9839
Epoch 3/5
8/8 [==============================] - 136s 17s/step - loss: 3.5802e-04 - accuracy: 1.0000 - val_loss: 7.7838e-05 - val_accuracy: 0.9839
Epoch 4/5
8/8 [==============================] - 151s 19s/step - loss: 1.8903e-04 - accuracy: 1.0000 - val_loss: 3.2829e-05 - val_accuracy: 0.9839
Epoch 5/5
8/8 [==============================] - 123s 15s/step - loss: 1.1132e-04 - accuracy: 1.0000 - val_loss: 1.8990e-05 - val_accuracy: 0.9839
